In [35]:
import xarray as xr
import os
import zarr

import sys
sys.path.append('')
import constants
sys.path.append(constants.MODULE_DIR)
import xarray_class_accessors as xca
import stats
import signal_to_noise as sn

In [29]:
FPATH = '/g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped/tas_mon_CCSM3_control_1530_g025.nc'

In [30]:
ds = xr.open_dataset(FPATH, use_cftime=True)

# Save as ZARR

In [10]:
os.path.dirname(FPATH)

'/g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped'

In [13]:
ROOT_DIR = '/g/data/w40/ab2313/PhD/longrunmip'
ZARR_PATH = os.path.join(ROOT_DIR, 'zarr_test')

In [11]:
os.mkdir(ZARR_PATH)

In [17]:
fname = os.path.basename(FPATH).replace('.nc', '')
fname

'tas_mon_CCSM3_control_1530_g025'

In [20]:
ds.to_zarr(os.path.join(ZARR_PATH, fname), mode='w')

# Open ZARR

In [33]:
ds_zar = xr.open_zarr(os.path.join(ZARR_PATH, fname), use_cftime=True)

# Speed Comparison

In [46]:
ds_zar['time'] = range(len(ds_zar.time.values))

In [47]:
ds['time'] = range(len(ds.time.values))

In [50]:
ds

<xarray.Dataset>
Dimensions:  (time: 1530, lon: 144, lat: 72)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 1523 1524 1525 1526 1527 1528 1529
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
Data variables:
    tas      (time, lat, lon) float32 234.3 234.0 234.0 ... 252.4 252.5 252.5

In [55]:
from importlib import reload

In [56]:
reload(stats)

<module 'stats' from '/home/563/ab2313/Documents/PhD/modules/stats.py'>

In [59]:
trend_ds = stats.trend_fit(ds.chunk({'time':-1}).dropna(dim='time'), method='lowess', lowess_window=10, 
               logginglevel='INFO')

- Detrending data using LOWESS with lowess_window=10


In [60]:
detrend_zarr = stats.trend_fit(ds_zar.chunk({'time':-1}), method='lowess', lowess_window=10)

In [61]:
detrend_zarr.compute()

<xarray.Dataset>
Dimensions:  (lat: 72, lon: 144, time: 1530)
Coordinates:
  * lat      (lat) float32 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 1523 1524 1525 1526 1527 1528 1529
Data variables:
    tas      (lat, lon, time) float64 234.9 234.8 234.7 ... 254.7 254.3 253.7

In [62]:
trend_ds.compute()

<xarray.Dataset>
Dimensions:  (lat: 72, lon: 144, time: 1530)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 1523 1524 1525 1526 1527 1528 1529
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
Data variables:
    tas      (lat, lon, time) float64 234.9 234.8 234.7 ... 254.7 254.3 253.7

In [37]:
sn_ds2 = sn.calculate_multi_window_signal_to_noise(ds_zar.chunk({'time':-1}), windows=[22, 24, 26],
                                                   logginglevel='ERROR', 
                                          detrend_kwargs={'method':'polynomial'}, parallel=False)

ValueError: method must be specified. Method options:  ['polynomial', 'lowess']